In [5]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from datasets import load_dataset, Dataset, load_metric
from torch.utils.data import DataLoader
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback
import torch
import numpy as np

os.getcwd()
#os.chdir('c:\\Users\\Kirchner\\Desktop\\Masterarbeit')

'/content'

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 3.2 MB/s 
     |████████████████████████████████| 120 kB 72.9 MB/s 
     |████████████████████████████████| 6.6 MB 57.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 2.7 MB/s 
     |████████████████████████████████| 115 kB 70.4 MB/s 
     |████████████████████████████████| 212 kB 69.8 MB/s 
     |████████████████████████████████| 127 kB 61.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:

import gc

gc.collect()

torch.cuda.empty_cache()



In [ ]:
# train_size = 50
# def read_annotations(txt_name, mode):
#     texts = []
#     labels = []

#     with open (txt_name, encoding = "utf-8", mode = "r+") as f:
#         for line in f.readlines():
#             texts.append(line.split("\t")[8])
#             if mode == "opinion":
#                 labels.append(line.split("\t")[5])
#             elif mode == "sentiment":
#                 labels.append(line.split("\t")[6])
#             else:
#                 print(f"There is no label for {mode}")

#     return texts,labels

In [ ]:
# meine_texte, meine_labels  = read_annotations("cleaned_annotated_data_training.txt", "sentiment")

# texts_train = meine_texte[:train_size]
# texts_test = meine_texte[train_size:]
# labels_train = meine_labels[:train_size]
# labels_test = meine_labels[train_size:]

# print(len(labels_train), len(labels_test))

In [3]:
### Here we import the tokenizer that we already trained if we change the flag to False, otherwise we take the default
### model
default = True
if os.path.isdir("meine_tokenizer") and not default:
    print("Loading finetuned tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("meine_tokenizer_10E_sentiment_500")
else:
    tokenizer = AutoTokenizer.from_pretrained("oliverguhr/german-sentiment-bert")

if os.path.isdir("Sentiment_Bert_mit_spiegel") and not default:
    print("Loading finetuned model...")
    model = AutoModelForSequenceClassification.from_pretrained("Sentiment_Bert_mit_spiegel_10E_sentiment_500")
else:
    model = AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert", 
                                                               num_labels=3,
                                                               
                                                               )

Downloading:   0%|          | 0.00/161 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
model.zero_grad()

In [6]:
import pandas as pd
import torch
from IPython.display import display
from scipy.stats import entropy

In [7]:
mein_dataframe1 = pd.read_csv("annotated_data/annotated_data_with_users_and_al_cleaned2check_final_topics.csv", header=None, encoding="ISO-8859-1")
mein_dataframe1.columns = ["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment",
                          "topic_comment", "Topic_article", "Comment", "Method"]
# mein_dataframe1.loc[:, "Comment"] = [x[:300] for x in mein_dataframe1["Comment"]]
# mein_dataframe1["Comment"]


In [8]:
data_text = []
data_labels = []
text_train = []
labels_train = []
text_test = np.array([])
labels_test = np.array([])


label_to_id = {"positive" : 0, "negative" : 1, "neutral" : 2}
id_to_label = {0:"positive", 1:"negative", 2:"neutral"}

### Here we create a training set that can be used to compare across different sizes of training data


mein_dataframe1.columns = ["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment",
                          "topic_comment", "Topic_article", "Comment", "Method"]

display(len(mein_dataframe1))
# split_index = int(len(mein_dataframe1)*0.8)
for index, row in mein_dataframe1.iterrows():
    data_text.append(row["Comment"])
    data_labels.append(row["Opinion"])

print(data_labels)   
# ### Define test and train set
# text_test = text_train[split_index+1:]
# text_train = text_train[:split_index]
# labels_test = labels_train[split_index+1:]
# labels_train = labels_train[:split_index]


### Replace label as int
for idx, labels in enumerate(data_labels):
    data_labels[idx] = label_to_id[labels]


# for idx, labels in enumerate(labels_test):
#     labels_test[idx] = label_to_id[labels]
    
    

# with open ("cleaned_annotated_data_training.txt", encoding="utf-8", mode="r+") as f:
#     training_data = f.readlines()
#     split_index = int(len(mein_dataframe1)*0.8)
    
#     ### The split on index 6 gives us Sentiment, split on 5 Opinion
    
#     ### The first 88 annotations had been annotated without a article topic
#     for line in training_data[:88]:
#         text_train.append(line.split("\t")[8])
#         labels_train.append(label_to_id[line.split("\t")[6]])
#     for line in training_data[89:split_index]:
#         text_train.append(line.split("\t")[9])
#         labels_train.append(label_to_id[line.split("\t")[6]])
#     for line in training_data[split_index+1:]:
#         text_test.append(line.split("\t")[9])
#         labels_test.append(label_to_id[line.split("\t")[6]])

def preprocess_function(examples):
    return tokenizer(examples["texts"], padding = "max_length", max_length=250, truncation=True)

def calculate_entropy(logits):
    probas = torch.nn.Softmax(dim=1)(torch.from_numpy(logits))
    samples_entropy = entropy(probas.transpose(0, 1).cpu())
    samples_entropy = torch.from_numpy(samples_entropy)
    return samples_entropy

def compute_metrics_accuracy(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        print("Predictions", predictions, "Labels", labels)
        return metric.compute(predictions=predictions, references=labels)

### Define splits for k fold cross validation
folds = StratifiedKFold(n_splits=5)

### Convert to numpy array
data_text = np.array([i for i in data_text])
data_labels = np.array([i for i in data_labels])



accuracies = []
metric = load_metric("accuracy")

for train_index, test_index in folds.split(data_text, data_labels):
    text_train, labels_train  = data_text[train_index], data_labels[train_index]
    text_test, labels_test  = data_text[test_index], data_labels[test_index]

    train_dict = {"texts": text_train, "labels" : labels_train}
    test_dict = {"texts": text_test, "labels" : labels_test}

    # print([len(z) for z in [text_train, text_test]])


    ### Remove newline characters
    for idx, text in enumerate(train_dict["texts"]):
        train_dict["texts"][idx] = text.rstrip("\n")

    for idx2, text2 in enumerate(test_dict["texts"]):
        test_dict["texts"][idx2] = text2.rstrip("\n")
        
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    meine_dataset_train = Dataset.from_dict(train_dict)

    mein_dataset_test = Dataset.from_dict(test_dict)

    tokenized_dataset_train = meine_dataset_train.map(preprocess_function, batched= True)
    tokenized_dataset_test = mein_dataset_test.map(preprocess_function, batched=True)

    training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.1,
    logging_steps=20,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True
    
    
    )


    trainer = Trainer(
        model= model,
        args=training_args,
        train_dataset=tokenized_dataset_train,
        eval_dataset=tokenized_dataset_test,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics = compute_metrics_accuracy,
        callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
       
    )

    trainer.train()
#     index_topk = torch.topk(calculate_entropy(trainer.predict(tokenized_dataset_test).predictions), 20).indices
#     print(index_topk)
#     for x in index_topk:
#         print(meine_dataset_train["texts"][x], "SATZ")
    accuracies.append(trainer.evaluate()['eval_accuracy'])
    model = AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert", 
                                                               num_labels=3,
                                                               hidden_dropout_prob=0.5
                                                               )

3996

['negative', 'positive', 'negative', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'negative', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'negative', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral', 'positive', 'negative', 'negative', 'negative', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'neutral', 'positive', 'negative', 'positive', 'neutral', 'neutral', 'positive', 'negative', 'neutral', 'positive', 'negative', 'negative', 'negative', 'neutral', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', '

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

Predictions [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 

Saving model checkpoint to ./results/checkpoint-12788
Configuration saved in ./results/checkpoint-12788/config.json
Model weights saved in ./results/checkpoint-12788/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-12788/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-12788/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-9591 (score: 1.1048941612243652).
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: texts. If texts are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 799
  Batch size = 16


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--oliverguhr--german-sentiment-bert/snapshots/c5c8dd0c5b966460dce1b7c5851bd90af1d2c6b6/config.json
Model config BertConfig {
  "_name_or_path": "oliverguhr/german-sentiment-bert",
  "_num_labels": 3,
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "germansentiment",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.5,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vo

Predictions [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 

All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at oliverguhr/german-sentiment-bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [ ]:
### Cross validated accuracy
print(len(text_train))
sum(accuracies)/len(accuracies)


1888


0.6148305084745763

In [ ]:
torch.cuda.get_device_name(0)

'GeForce GTX TITAN X'

In [9]:

model.save_pretrained("Sentiment_Bert_Opinion")
tokenizer.save_pretrained("meine_tokenizer_Sentiment_Bert_Opinion")


Configuration saved in Sentiment_Bert_Opinion/config.json
Model weights saved in Sentiment_Bert_Opinion/pytorch_model.bin
tokenizer config file saved in meine_tokenizer_Sentiment_Bert_Opinion/tokenizer_config.json
Special tokens file saved in meine_tokenizer_Sentiment_Bert_Opinion/special_tokens_map.json


('meine_tokenizer_Sentiment_Bert_Opinion/tokenizer_config.json',
 'meine_tokenizer_Sentiment_Bert_Opinion/special_tokens_map.json',
 'meine_tokenizer_Sentiment_Bert_Opinion/vocab.txt',
 'meine_tokenizer_Sentiment_Bert_Opinion/added_tokens.json',
 'meine_tokenizer_Sentiment_Bert_Opinion/tokenizer.json')